# Passing Structures that Contains SharedArrays

What happens when we have non-shared structures that contains SharedArrays?  I would think that these structures would be just containers of local variables.  This would be a fairly common case becase a single process may load the data but shares it with other child processes before a heavy computation process begins.

Note that this is not the same problem of loading data in parallel by workers.  In that case, the container should be set up via a `@everywhere` macro with empty SharedArray's and then woker processes can just fill in the blanks.  Let's do that at a separate notebook.

In [1]:
procs()

1-element Array{Int64,1}:
 1

In [2]:
addprocs(2)
workers()

2-element Array{Int64,1}:
 2
 3

In [3]:
# get the worker id and their respective pid's
[fetch(@spawnat i (myid(), getpid())) for i in workers()]

2-element Array{Tuple{Int64,Int32},1}:
 (2, 26848)
 (3, 26849)

In [4]:
# create a dict that contains SharedArray's
d = Dict(
    :a => SharedArray{Float64}(rand(1000)),
    :b => SharedArray{Float64}(rand(1000))
    )

Dict{Symbol,SharedArray{Float64,1}} with 2 entries:
  :a => [0.287936, 0.942762, 0.203745, 0.445032, 0.749347, 0.295443, 0.143348, …
  :b => [0.956132, 0.0326915, 0.988323, 0.0389778, 0.70849, 0.269639, 0.44612, …

The worker processes should have access to the elements inside the dictionary but not the dictionary itself.... sounds strange but let's test it out below.

In [5]:
@everywhere function displaydvar() 
    global d
    display(d)
end

In [6]:
fetch(remotecall(displaydvar, workers()[1]))

LoadError: [91mOn worker 2:
[91mUndefVarError: d not defined[39m
displaydvar at ./In[5]:3
#103 at ./distributed/process_messages.jl:264 [inlined]
run_work_thunk at ./distributed/process_messages.jl:56
run_work_thunk at ./distributed/process_messages.jl:65 [inlined]
#96 at ./event.jl:73[39m

In [7]:
fetch(remotecall(displaydvar, workers()[2]))

LoadError: [91mOn worker 3:
[91mUndefVarError: d not defined[39m
displaydvar at ./In[5]:3
#103 at ./distributed/process_messages.jl:264 [inlined]
run_work_thunk at ./distributed/process_messages.jl:56
run_work_thunk at ./distributed/process_messages.jl:65 [inlined]
#96 at ./event.jl:73[39m

In [8]:
# define a function that takes the container `d` as an argument
@everywhere sum_a(d) = sum(d[:a])

In [9]:
# let's test it by running it on pid 4
fetch(remotecall(sum_a, workers()[1], d))

497.1482241515213

In [10]:
fetch(remotecall(sum_a, workers()[2], d))

497.1482241515213

In [11]:
rmprocs(workers())
procs()

1-element Array{Int64,1}:
 1